### 구현 방식
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸린다.
3. (쪼갠 문서를) 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [71]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
loader = Docx2txtLoader("./tax_with_markdown.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [72]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [73]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-markdown-index"
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 추가한 이후에는 --> from_existing_index() 사용
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [83]:
# query = "연봉 5,000만 원인 직장인의 소득세는 얼마인가요?"
query = "연봉 5,000만 원인 거주자의 종합소득세는 얼마인가요?"

In [ ]:
retriever = database.as_retriever(search_kwargs={'k': 2})
retriever.invoke(query)

In [85]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [86]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [87]:
from langchain.chains import RetrievalQA # Question, Answer chain

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt" : prompt}
)

# 사용자가 넘겨주는 query 중 (직장인 => 거주자)로 변경해주는 chain 을 추가해보자
# [as-is] ai_message = qa_chain({"query" : query})

In [105]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

# prompt 작성
# 참고로 prompt는 영어로 작성하는게 더 효과적이라고 한다.
# 강의에서는 한국어로 작성해서 나도 한국어로 작성
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요

    사전: {dictionary}
    
    질문 : {{question}}
""")

# prompt를 llm에 넘겨주고 그 결과를 StrOutputParser를 이용해 파싱
dictionary_chain = prompt | llm | StrOutputParser()

In [ ]:
# dictionary_chain 잘 되는지 확인
new_question = dictionary_chain.invoke({"question": query})
new_question

In [107]:
"""
    ai_message = qa_chain({"query" : query})
    - 기존에 qa_chain 호출하는 모습

    tax_chain = {"query" : dictionary_chain} | qa_chain
    - dictionary_chain의 결과를 "query"라는 key의 value로 담아
    - qa_chain에 넘겨준다.

    [결론] qa_chain이 받는 query는 dictionary_chain의 결과이다.
    [결론] {"query" : query} ≒ {"query" : dictionary_chain}
"""

# dictionary_chain의 결과를 qa_chain에 넘겨준다.
tax_chain = {"query" : dictionary_chain} | qa_chain

In [ ]:
# qa_chain에 query를 넘기려면 dictionary_chain이 먼저 실행돼야 하기에
# {"question": query} 식으로 넣어준다.
ai_response = tax_chain.invoke({"question": query})
ai_response